<a href="https://colab.research.google.com/github/dymiyata/erdos2023_million_playlist_challenge/blob/master/matrix_factorization/matrix_factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import json
import copy
import random
from numba import njit

import sqlalchemy as db

from time import time

from sklearn.model_selection import train_test_split

# Define functions

Define a function to load the SQL database into R_list

In [2]:
def make_R_list_sql(conn, pid_limit=None, progress=5.):
    if pid_limit is None:
        # Get number of pairings
        results = conn.execute(db.text("SELECT COUNT(*) FROM pairings"))
        N = results.fetchall()[0][0]
        
        # Get columns one by one
        results = conn.execute(db.text("SELECT pid, tid FROM pairings"))
    else:
        # Get number of pairings
        results = conn.execute(db.text(f"SELECT COUNT(*) FROM pairings WHERE pid<{pid_limit}"))
        N = results.fetchall()[0][0]
        
        # Get columns one by one
        results = conn.execute(db.text(f"SELECT pid, tid FROM pairings WHERE pid<{pid_limit}"))
    
    R_list = np.empty((N,2))
    for i in range(N):
        
        if not progress is None:
            # Show progress
            if i % np.round(N*progress/100) == 0:
                print('{:.2f}%'.format(i/N*100))
        
        row = results.fetchone()
        R_list[i,0] = int(row[0])
        R_list[i,1] = int(row[1])
    
    return R_list

The functions for running gradient descent and computing the error function

In [4]:
# use gradient descent to minimize MSE (with l2 regularization)
def update_params_loop(R_list, P, Q, alpha, llambda):
    newP = P
    newQ = Q
    m , f = np.shape(P)
    n = np.shape(Q)[0]

    for u,i in R_list:
        newP[u,:] -= alpha * 2 * (P[u,:] @ Q[i,:] - 1) * Q[i,:]
        newQ[i,:] -= alpha * 2 * (P[u,:] @ Q[i,:] - 1) * P[u,:]
    for u in range(m):
        newP[u,:] -= alpha * 2 * llambda * P[u,:]
    for i in range(n):
        newQ[i,:] -= alpha * 2 * llambda * Q[i,:]
    return (newP, newQ)

# use gradient descent with where R_list has triples (u,i,score)
def update_params_loop_score(R_list, P, Q, alpha, llambda):
    newP = P
    newQ = Q
    m , f = np.shape(P)
    n = np.shape(Q)[0]

    for u,i, score in R_list:
        newP[u,:] -= alpha * 2 * (P[u,:] @ Q[i,:] - score) * Q[i,:]
        newQ[i,:] -= alpha * 2 * (P[u,:] @ Q[i,:] - score) * P[u,:]
    for u in range(m):
        newP[u,:] -= alpha * 2 * llambda * P[u,:]
    for i in range(n):
        newQ[i,:] -= alpha * 2 * llambda * Q[i,:]
    return (newP, newQ)

# Run an epoch of gradient descent where the iterations parameter is the number of iterations.
@njit
def run_epoch(R_array, P, Q, alpha, llambda, iterations):
    oldP = P.copy()
    oldQ = Q.copy()
    f = np.shape(P)[1]

    for iter in range(iterations):
        newP = oldP.copy()
        newQ = oldQ.copy()
        for u,i in R_array:
            dotprod = 0
            for feature in range(f):
                dotprod += oldP[u,feature] * oldQ[i,feature]
            error = dotprod - 1

            for feature in range(f):
                pf = oldP[u,feature]
                qf = oldQ[i,feature]
                newP[u,feature] -= alpha * (error * qf + llambda * pf) 
                newQ[i,feature] -= alpha * (error * pf + llambda * qf)
        oldP = newP
        oldQ = newQ
    return newP, newQ


#runs the gradient descent loop in batches
def gd_batch(R_list, P, Q, alpha, llambda, batch_num, iterations, R = None, verbose=False):
    #make copies of P and Q
    P_current = P.copy()
    Q_current = Q.copy()

    #shuffle R_list
    #divide R_list into batch_num subsets
    random.shuffle(R_list)
    batch_size = int(np.ceil(len(R_list)/batch_num))
    R_batch = [R_list[i:i+batch_size] for i in range(0,len(R_list), batch_size) ]

    #loop over total iterations
    for i in range(iterations):
        #if verbose == true print out error function
        if verbose:
            print(f'Step {i*batch_num}: Error function={error_function(R_list,R , P_current, Q_current)}')
        #loop over batch_num
        for batch in R_batch:
            #run update_param_loop on batch
            P_current , Q_current = update_params_loop(batch, P_current, Q_current, alpha, llambda)

    return (P_current , Q_current)

#error function without l2 normalization factor
def error_function( R_list,R, P , Q ):
    result = 0
    #sum over R_list
    for row, col in R_list:
        result = result + (R[row,col] - P[row,:]@Q[col,:])**2

    return result

@njit
def error_function_l2( R_list, P , Q, llambda):
    result = 0
    #sum over R_list
    for row, col in R_list:
        result = result + (1 - P[row,:]@Q[col,:])**2
    result += llambda * (np.linalg.norm(P)**2 + np.linalg.norm(Q)**2)
    return result

The function for obtaining the playlist vector that minimizes the cost function for a fixed $Q$ from a list of track id's.

In [5]:
def new_user_vec(tid_list, Q, llambda):
    Y = Q[tid_list,:]
    f = np.shape(Q)[1]
    d = len(tid_list)
    vec = np.linalg.inv(np.transpose(Y) @ Y + llambda * np.identity(f)) @ np.transpose(Y) @ np.ones((d,1))
    return np.transpose(vec)

# Training the Model

The next two cells read in data and creates the list of data points.  Ideally this will be done by querying the SQL database.

In [6]:
db_path = "sqlite:///../spotify_sql_with_tid_0.db"
engine = db.create_engine(db_path)
conn = engine.connect()

In [7]:
time_0 = time()
R_list = make_R_list_sql(conn, pid_limit=10000, progress=10)
time_end = time()

print('{:.3f} sec'.format(time_end-time_0))
print('{:.3f} minutes'.format((time_end-time_0)/60))
print()

num_playlists = len(np.unique(R_list[:,0]))
print(f'We have {num_playlists} playlists')

0.00%
10.00%
20.00%
30.00%
40.00%
50.00%
60.00%
70.00%
80.00%
90.00%
100.00%
8.041 sec
0.134 minutes

We have 10000 playlists


Create train-test split

In [8]:
# Percentage of the total database to reserve for validation and testing
val_size_abs = 0.15
test_size    = 0.15
shuffle = True

# Note: the first pid_train contains (1-test_size) percent of the data.
# We need to use val_size so that val_size*(1-test_size) = val_size_abs.
val_size = val_size_abs/(1-test_size)
pid_train, pid_test = train_test_split(np.arange(num_playlists), test_size=test_size,
                                       shuffle=shuffle, random_state=11)
pid_train, pid_val  = train_test_split(pid_train, test_size=val_size,
                                       shuffle=shuffle, random_state=11)

In [9]:
R_list_train = R_list[ np.isin(R_list[:,0], pid_train), :]
R_list_val   = R_list[ np.isin(R_list[:,0], pid_val),   :]
R_list_test  = R_list[ np.isin(R_list[:,0], pid_test),  :]

# Store the track id of songs in the train/val/test sets
tid_train = list(np.unique( R_list_train[:,1] ))
tid_val   = list(np.unique( R_list_val[:,1]   ))
tid_test  = list(np.unique( R_list_test[:,1]  ))

num_songs = len(tid_train)

## Example of training the algorithm

Next, we specify the number of features and create matrices $P$ and $Q$ whose entries are randomly taken from a normal distribution with $\mu = 0$ and $\sigma = 0.1$.

In [ ]:
f = 10 # number of latent features
# num_songs = max(reverse_track_dict.keys()) + 1
num_songs = len(tid_train)
num_playlists = len(pid_train)

# initialize random values for matrices P and Q. Entries are between -1 and 1
P = np.random.normal(0, 0.1, (num_playlists, f))
Q = np.random.normal(0, 0.1, (num_songs, f))

In the following cell, we run the gradient descent algorithm and store the resulting matrices in P_trained and Q_trained.

In [ ]:
# Run gradient descent algorithm with alpha = 0.001, llambda = 0.005 for 100 iterations
P_trained, Q_trained = run_epoch(R_list_train, P, Q, 0.001, 0.005, 100)

### Example of computing error on non-training data

Now, to demonstrate how to compute error for a new collection of playlists, let's first read in an extra json file's worth of data (I could've just read in the single extra file instead of all original files again but I was too lazy to change the read_data function). Again this step should utilize the SQL database.

**Note:** I changed R_list_new into R_list_test during the train-val-test split.

For our fixed Q, given a list of (new) playlist ids, we can compute the P matrix that minimizes the 

In [ ]:
# THIS FUNCTION, (as well as new_user_vec) CAN AND SHOULD BE OPTIMIZED TO USE numba.  
# For the sake of time, I'll leave it as is for now
# If we can avoid using lists (like tid_list), then things will be numba compatible. 
def make_Pval(new_pids, R_list_new, Q, llambda, tid_known=tid_train):
    num_songs , f = np.shape(Q)
    P_val = np.zeros((len(new_pids), f))
    count = 0 # keeps track of where in pid_list we are
    
    # Remove tracks that we don't recognize
    R_list_new = R_list_new[ np.isin(R_list_new[:,1], tid_known), : ]
    
    for pid in new_pids:
        # create list of tracks in the playlist
        # Remember: R_list_new has two columns: 0 is pid, 1 is tid
        tid_list = R_list_new[ R_list_new[:,0]==pid, 1]
        
        # With repetition
        # tid_list = list(tid_list)
        # tid_list = [tid for tid in tid_list if tid in tid_known]
        
        # Without repetition
        # tid_list = list( set(tid_list).intersection(tid_known) )

        # x is the row of Pval corresponding to this pid
        x = new_user_vec(tid_list, Q_trained, llambda)
        for feature in range(f):
            P_val[count, feature] = x[0,feature]
        count += 1
        
    return P_val

Pval = make_Pval(pid_test, R_list_test, Q_trained, 0.005)

Next, we compute the error function on this set. Note, we can't use the function defined at the top of this document because now the pid does not correspond to the index of the column of Pval.

In [ ]:
def val_error(R_list_new, pid_list, Pval, Q, llambda, tid_known=tid_train):
    _, f = np.shape(Q)
    result = 0
    
    # Remove tracks that we don't recognize
    R_list_new = R_list_new[ np.isin(R_list_new[:,1], tid_known), : ]
    
    for pid, tid in R_list_new:
        result += (1 - Pval[pid_list.index(pid), :] @ Q[tid, :])
    
    result += llambda * (np.linalg.norm(Pval)**2)
    return result

val_error(R_list_test, pid_test, Pval, Q_trained, 0.005)

# Find best hyperparameters using grid search

In [10]:
# Hyperparameter tuning using grid search
# Specify the number of latent features (f), learning rate (alpha), and regularization parameter (llambda)
f_values = [5, 10, 15]
alpha_values = [0.001, 0.01, 0.1]
llambda_values = [0.001, 0.01, 0.1]

NUM_ITERATIONS = 100

nF = len(f_values)
nA = len(alpha_values)
nL = len(llambda_values)

time_start = time()
best_error = float('inf')
best_hyperparameters = None
for idf in range(nF):
    f = f_values[idf]
    for ida in range(nA):
        alpha = alpha_values[ida]
        for idl in range(nL):
            llambda = llambda_values[idl]
            
            time_0 = time()
            
            # Initialize random values
            P_initial = np.random.normal(0, 0.1, (num_playlists, f))
            Q_initial = np.random.normal(0, 0.1, (num_songs, f))
            
            # Obtain P, Q with the chosen hyperparameters
            P_trained, Q_trained = run_epoch(R_list_train, P=P_initial, Q=Q_initial, alpha=alpha, llambda=llambda, iterations=NUM_ITERATIONS)
            
            # Check results against the validation set
            P_val = make_Pval(pid_val, R_val, Q_trained, llambda)
            current_error = error_function_l2(R_val, P_val, Q_trained, llambda)
            
            # Store the best hyperparameters
            if current_error < best_error:
                best_error = current_error
                best_hyperparameters = {'f': f, 'alpha': alpha, 'llambda': llambda}
            
            time_t = time()
            print(f'({idf},{ida},{idl})/({nF},{nA},{nL}):')
            print('{:.3f} sec'.format(time_t-time_0))
            print('{:.3f} minutes'.format((time_t-time_0)/60))
            print()

time_end = time()
print('Total time:')
print('{:.3f} sec'.format(time_end-time_start))
print('{:.3f} minutes'.format((time_end-time_start)/60))
print()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(float64, 2d, C), Tuple(float64, int64))
 
There are 22 candidate implementations:
[1m      - Of which 20 did not match due to:
      Overload of function 'getitem': File: <numerous>: Line N/A.
        With argument(s): '(array(float64, 2d, C), Tuple(float64, int64))':[0m
[1m       No match.[0m
[1m      - Of which 2 did not match due to:
      Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 209.
        With argument(s): '(array(float64, 2d, C), Tuple(float64, int64))':[0m
[1m       Rejected as the implementation raised a specific error:
         NumbaTypeError: [1mUnsupported array index type float64 in Tuple(float64, int64)[0m[0m
  raised from /home/MRGomez/anaconda3/envs/audio/lib/python3.11/site-packages/numba/core/typing/arraydecl.py:102
[0m
[

In [18]:
# Initialize random values
P_initial = np.random.normal(0, 0.1, (num_playlists, f))
Q_initial = np.random.normal(0, 0.1, (num_songs, f))

# Obtain P, Q with the chosen hyperparameters
P_trained, Q_trained = run_epoch(R_list_train, P=P_initial, Q=Q_initial, alpha=alpha, llambda=llambda, iterations=NUM_ITERATIONS)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(float64, 2d, C), Tuple(float64, int64))
 
There are 22 candidate implementations:
[1m      - Of which 20 did not match due to:
      Overload of function 'getitem': File: <numerous>: Line N/A.
        With argument(s): '(array(float64, 2d, C), Tuple(float64, int64))':[0m
[1m       No match.[0m
[1m      - Of which 2 did not match due to:
      Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 209.
        With argument(s): '(array(float64, 2d, C), Tuple(float64, int64))':[0m
[1m       Rejected as the implementation raised a specific error:
         NumbaTypeError: [1mUnsupported array index type float64 in Tuple(float64, int64)[0m[0m
  raised from /home/MRGomez/anaconda3/envs/audio/lib/python3.11/site-packages/numba/core/typing/arraydecl.py:102
[0m
[